In [1]:
# Imports
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# import the world bank utility functions. It requires changing the path to import properly in Python
import sys
sys.path.append('../utility_functions')

from world_bank_utility_functions import (get_world_bank_indicator_name_from_code, 
                                          get_world_bank_indicator_definition_from_code)

# for displaying text with formatting later
from IPython.core.display import display, HTML

/tmp/ipykernel_234387/3840827134.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Select data
datasets = ["../OECD/Cleaned/HEALTH_MERGED_Threshold_80_n3.csv", "../WorldBankDatasets/Cleaned/AllMerged_Threshold_85_n3_MMRTNE.csv"]

OECD = False  # False for WB data

if OECD:
    df = pd.read_csv(datasets[0])
    mm_ind = "MATIMATM"
else:
    df = pd.read_csv(datasets[1])
    mm_ind = "SH.STA.MMRT.NE"

In [3]:
df

,Country,Year,BM.GSR.ROYL.CD,BX.GSR.ROYL.CD,GB.XPD.RSDV.GD.ZS,IP.JRN.ARTC.SC,IP.PAT.NRES,IP.PAT.RESD,SP.POP.SCIE.RD.P6,SP.POP.TECH.RD.P6,...,UIS.T.3.F,UIS.THAGE.0,UIS.THAGE.3.A.GPV,UIS.THAGE.4.A.GPV,UIS.THDUR.0,UIS.THDUR.4.A.GPV,UIS.XSPENDP.56.FDPUB.FNCAP,UIS.XSPENDP.56.FDPUB.FNCUR,UIS.XSPENDP.56.FDPUB.FNNONS,UIS.XSPENDP.56.FDPUB.FNS
0,AUS,1997,0.016586,0.003082,0.412299,0.050619,0.049629,0.004538,0.437652,0.211022,...,0.017427,0.400000,0.666667,0.0,0.333333,0.0,0.211677,0.788317,0.307628,0.626075
1,AUS,2000,0.016077,0.003387,0.318885,0.054746,0.064698,0.004972,0.440204,0.202796,...,0.017135,0.800000,0.666667,0.0,0.000000,0.0,0.176835,0.823146,0.302216,0.653377
2,AUS,2003,0.020823,0.003873,0.412299,0.059787,0.061806,0.006237,0.437652,0.211022,...,0.017472,0.200000,0.666667,0.0,0.500000,0.0,0.211677,0.788317,0.307628,0.626075
3,AUS,2006,0.029817,0.005513,0.452984,0.079045,0.074667,0.007319,0.539751,0.256436,...,0.018845,0.200000,0.666667,0.0,0.500000,0.0,0.197271,0.802729,0.317720,0.623900
4,AUS,2011,0.056962,0.008149,0.465026,0.104943,0.074593,0.006147,0.508147,0.296302,...,0.251432,0.200000,0.666667,0.0,0.500000,0.0,0.260925,0.739075,0.302946,0.600949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,USA,2010,0.431172,0.816016,0.575456,0.955735,0.800173,0.624674,0.492739,0.121917,...,0.913240,0.066667,0.333333,0.4,0.722222,0.0,0.156037,0.843963,0.285341,0.684574
1151,USA,2011,0.456073,0.919857,0.582051,0.987737,0.824615,0.639577,0.511049,0.121917,...,0.913240,0.066667,0.333333,0.4,0.722222,0.0,0.156037,0.843963,0.285341,0.684574
1152,USA,2014,0.520510,1.000000,0.572387,0.999216,0.946693,0.735989,0.534292,0.121917,...,0.982993,0.066667,0.333333,0.4,0.722222,0.0,0.199686,0.800314,0.280513,0.663046
1153,USA,2015,0.487474,0.955070,0.585766,0.999364,0.970446,0.744350,0.542535,0.121917,...,1.000000,0.066667,0.333333,0.4,0.722222,0.0,0.182733,0.817267,0.284023,0.669622


In [4]:
# mapping world bank codes to definitions
mapping_world_bank_to_names_and_definitions_df = pd.read_csv('../WorldBankDatasets/Cleaned/World_Bank_Indicator_Definition_Info.csv',
                                                             delimiter='\t')

In [5]:
mapping_world_bank_to_names_and_definitions_df

,indicator_code,indicator_name,long_definition
0,Country,indicator name missing,definition missing
1,Year,indicator name missing,definition missing
2,SH.ANM.ALLW.ZS,Prevalence of anemia among women of reproducti...,Prevalence of anemia among women of reproducti...
3,SH.ANM.CHLD.ZS,Prevalence of anemia among children (% of chil...,"Prevalence of anemia, children ages 6-59 month..."
4,SH.ANM.NPRG.ZS,Prevalence of anemia among non-pregnant women ...,"Prevalence of anemia, non-pregnant women, is t..."
...,...,...,...
791,SE.TER.CUAT.MS.MA.ZS,"Educational attainment, at least Master's or e...",The percentage of population ages 25 and over ...
792,SG.GEN.MNST.ZS,Proportion of women in ministerial level posit...,Women in ministerial level positions is the pr...
793,SG.POP.MIGR.FE.ZS,Female migrants (% of international migrant st...,Percentage of female migrants out of total int...
794,SH.MMR.WAGE.ZS,Maternal leave benefits (% of wages paid in co...,Maternity leave benefits refers to the total p...


## Data Preparation

In [6]:
# Reduce to only gender indicators

#if not OECD:
#    gender_ind = pd.read_csv('../WorldBankDatasets/Gender_WorldBankData.csv').columns
#    df = df.drop(columns=[col for col in df if col not in gender_ind])

#df.head()

In [7]:
# Select year
year = 2015
df = df[df["Year"] == year]
df.drop("Year", axis=1, inplace=True)

# Scale predictor, which is currently unscaled
scaler = MinMaxScaler()
df[mm_ind] = scaler.fit_transform(df[mm_ind].to_numpy().reshape(-1, 1))

# Drop country
X = df.drop(columns=['Country'])

/tmp/ipykernel_234387/2488832800.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("Year", axis=1, inplace=True)
/tmp/ipykernel_234387/2488832800.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[mm_ind] = scaler.fit_transform(df[mm_ind].to_numpy().reshape(-1, 1))


## Dimensionality Reduction

# PCA

In [8]:
pca = PCA(n_components=2, random_state=42)
pca.fit(X)

print("Number of PCs: {}".format(len(pca.explained_variance_ratio_)))
print("Explained variation per PC: {}".format(pca.explained_variance_ratio_))
print("Sum of explained variation: {}".format(pca.explained_variance_ratio_.sum()))

Number of PCs: 2
Explained variation per PC: [0.423584   0.10788109]
Sum of explained variation: 0.5314650840062904


# Now to show you detailed information of the features that contribute to the PC analysis

In [9]:
def sort_loadings_for_principal_component(loadings, X=X):
    feature_names = X.columns
    
    # make a series of the current pca loadings. Index is the column name (feature) and value is the loading value
    feature_names_series = pd.Series(loadings, index=feature_names)

    # take the absolute value of the series
    feature_names_series_abs = feature_names_series.abs()

    # finally sort descending. The higher the number, the higher the correlatino and explanatory value of the feature for the principal component
    pca_component_explanatory_features_sorted = feature_names_series_abs.sort_values(ascending=False)
    
    return pca_component_explanatory_features_sorted

In [10]:
def get_most_important_features_from_pca_loadings (pca_loadings, n_most_important_features_from_loading = 5, 
                                                   includeName = True, includeDefintion = True):
    pca_loadings_explanations = {}

    cur_loadings_sorted_with_n_most_important_features = sort_loadings_for_principal_component(pca_loadings).iloc[
        0:n_most_important_features_from_loading]

    feature_codes = []
    feature_values = []

    feature_names = []
    feature_definitions = []

    for feature_code_index, feature_value in zip (cur_loadings_sorted_with_n_most_important_features.index,
                                                  cur_loadings_sorted_with_n_most_important_features.values):
        feature_codes.append(feature_code_index)
        feature_values.append(feature_value)

        if (includeName):
            cur_name = get_world_bank_indicator_name_from_code(feature_code_index)
            # print('cur_name is', cur_name)
            feature_names.append(cur_name)
        
        if (includeDefintion):
            cur_definition  = get_world_bank_indicator_definition_from_code(feature_code_index)
            feature_definitions.append(cur_definition)

    # pca_loadings_explanations[f"top_{n_most_important_features_from_loading}_most_important_feature_values"] = feature_values
    pca_loadings_explanations["most_important_features_values"] = feature_values
    pca_loadings_explanations["most_important_features_world_bank_codes"] = feature_codes
    pca_loadings_explanations["most_important_features_world_bank_names"] = feature_names
    pca_loadings_explanations["most_important_features_world_bank_definitions"] = feature_definitions

    return pca_loadings_explanations

In [11]:
# this function will show the top most important components explaining the variance for each of the pcs 
# give a given percentage of variance to explain
def pca_with_detailed_variance_and_components_info(percentage_of_variance_to_explain, X=X, n_most_important_features_from_loading=5, includeValues = False,
                                                 includeDefinition=False):
    pca = PCA(n_components=percentage_of_variance_to_explain, random_state=42)
    pca.fit(X)
    pca_loadings = pca.components_

    loadings_information = []
    for loading in pca_loadings:
        cur_loading_features_info = get_most_important_features_from_pca_loadings(loading, n_most_important_features_from_loading)
        loadings_information.append(cur_loading_features_info)
    
    num_principal_components = len(pca.explained_variance_ratio_)

    print(f'in order to explain {percentage_of_variance_to_explain * 100} percentage of the variance with pca it requires')
    print(f'{num_principal_components} principal components')
    print(f'the most important feature names, for the top {n_most_important_features_from_loading} from each PC, are\n')

    # print the most important value names
    most_important_feature_names_explaining_variance_with_pca = []
    for i in range(0, len(loadings_information)):
        # print('\n')
        print('---------')
        print(f'For Principal Component {i+1}, the most important features names are:\n')
        # print('\n')
        cur_loadings_info = loadings_information[i]

        cur_loadings_names = cur_loadings_info['most_important_features_world_bank_names']
        cur_loadings_values = cur_loadings_info['most_important_features_values']
        cur_loadings_codes = cur_loadings_info['most_important_features_world_bank_codes']
        cur_loadings_definitions = cur_loadings_info['most_important_features_world_bank_definitions']

        for code, value, name, definition in zip(cur_loadings_codes, cur_loadings_values, cur_loadings_names, cur_loadings_definitions):
            print(name)
            
            # uncomment for testing to debug
            # print(code)

            if (includeValues):
                print(f"with value {value}")

            if (includeDefinition):
                print(definition)
        print('\n')

## Modify variance_percentage_to_explain below for detailed information based on what percentage of the variance in principal components you want to explain maternal mortality. (see section below for just the top feature per PC)

In [12]:
# pca_with_detailed_variance_and_components_info(0.8)
# pca_with_detailed_variance_and_components_info(0.9, includeValues=True)
num_top_features_per_pc = 5
pca_with_detailed_variance_and_components_info(0.9, includeValues=True, 
                                               n_most_important_features_from_loading = num_top_features_per_pc)

in order to explain 90.0 percentage of the variance with pca it requires
18 principal components
the most important feature names, for the top 5 from each PC, are

---------
For Principal Component 1, the most important features names are:

There are periods of absence due to childcare accounted for in pension benefits (1=yes; 0=no)
with value 0.07151985342348981
Law mandates equal remuneration for females and males for work of equal value (1=yes; 0=no)
with value 0.06792436978253891
Secondary education, vocational pupils
with value 0.06718282519487066
Enrolment in lower secondary education, female (number)
with value 0.06649493079538345
Enrolment in lower secondary education, both sexes (number)
with value 0.06545147071788737


---------
For Principal Component 2, the most important features names are:

The age at which men and women can retire with full pension benefits is the same (1=yes; 0=no)
with value 0.15122940803154442
Domestic general government health expenditure per capita,

In [13]:
# returns a list of the top feature name for each principal component
def get_top_feature_for_each_pc(percentage_of_variance_to_explain, X=X, n_most_important_features_from_loading=5):
    pca = PCA(n_components=percentage_of_variance_to_explain, random_state=42)
    pca.fit(X)
    pca_loadings = pca.components_

    loadings_information = []
    for loading in pca_loadings:
        cur_loading_features_info = get_most_important_features_from_pca_loadings(loading, n_most_important_features_from_loading)
        loadings_information.append(cur_loading_features_info)
    
    top_feature_name_per_pc = []
    for i in range(0, len(loadings_information)):
        cur_loadings_info = loadings_information[i]

        cur_features_names = cur_loadings_info['most_important_features_world_bank_names']
        cur_top_feature = cur_features_names[0]
        top_feature_name_per_pc.append(cur_top_feature)
        # cur_loadings_values = cur_loadings_info['most_important_features_values']
        # cur_loadings_codes = cur_loadings_info['most_important_features_world_bank_codes']
        # cur_loadings_definitions = cur_loadings_info['most_important_features_world_bank_definitions']
    
    return top_feature_name_per_pc

## Modify variance_percentage_to_explain below based on what percentage of the variance in principal components. You may find the results interesting!

In [14]:
# below starts to use html formatting

In [15]:
# variance_percentage_to_explain = 0.2
# variance_percentage_to_explain = 0.3
# variance_percentage_to_explain = 0.5
# variance_percentage_to_explain = 0.8
# variance_percentage_to_explain = 0.85
# variance_percentage_to_explain = 0.9
variance_percentage_to_explain = 0.95

top_feature_names_for_pc_loadings = get_top_feature_for_each_pc(variance_percentage_to_explain)

# print(f"""Assuming you want to explain {variance_percentage_to_explain * 100} percent of the PC variation, 
# for maternal morality, the top feature names that explain this per principal component are:\n""")

display( HTML(f"""<h2>Assuming you want to explain {variance_percentage_to_explain * 100} percent of the PC variation, 
              the top feature names that explain this per principal component are:</h2>\n"""))

for i in range(0, len(top_feature_names_for_pc_loadings)):
    #print(f"for PC {i+1} the top explaining feature is: {top_feature_names_for_pc_loadings[i]}")
    # print(f"PC {i+1}: {top_feature_names_for_pc_loadings[i]}")
    display( HTML( f"<strong>PC {i+1}:</strong> {top_feature_names_for_pc_loadings[i]}" ) )
    # print("\n")